In [87]:
import pandas as pd
from prophet import Prophet


In [88]:
# Load the Excel file
df = pd.read_excel('/content/drive/MyDrive/budgets/SalesBudgetnv1.xlsx')

In [89]:
# Ensure the 'Date' column is in datetime format
df['Date'] = pd.to_datetime(df['Date'])

In [90]:
# Set the 'Date' column to the first day of the month at 00:00:00
df['Date'] = df['Date'] + pd.offsets.MonthBegin(0)

In [91]:
# Rename columns to match Prophet's requirements
df = df.rename(columns={'Date': 'ds', 'Budget': 'y'})

In [92]:
# Initialize and fit the model
model = Prophet()
model.fit(df)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpc4xqd9ry/bava4jv2.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpc4xqd9ry/92eho6o0.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=85588', 'data', 'file=/tmp/tmpc4xqd9ry/bava4jv2.json', 'init=/tmp/tmpc4xqd9ry/92eho6o0.json', 'output', 'file=/tmp/tmpc4xqd9ry/prophet_modeld2ls5ipu/prophet_model-20231129133654.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
13:36:54 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
13:36:54 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [93]:
# Create a dataframe with future dates for prediction (12 months)
future = model.make_future_dataframe(periods=12, freq='M')

In [94]:
# Set the future dates to the first day of the month at 00:00:00
future['ds'] = future['ds'] + pd.offsets.MonthBegin(0)

In [95]:
# Generate predictions for the future dates
forecast = model.predict(future)

In [96]:
# Extract the predicted values for the future months
predicted_values = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(12)


In [97]:
# Merge the predicted values with the original dataframe
result_df = pd.merge(df, predicted_values, how='outer', left_on='ds', right_on='ds')


In [98]:
# Replace missing values in the 'Budget' column with the predicted values
result_df['y'] = result_df['yhat'].combine_first(result_df['y'])


In [99]:
# Select the necessary columns and rename them
result_df = result_df[['ds', 'y']].rename(columns={'ds': 'Date', 'y': 'Budget'})

In [100]:
# Save the result to a new Excel file or overwrite the existing one
result_df.to_excel('/content/drive/MyDrive/budgets/SalesBudgetnv2.xlsx', index=False)